In [1]:
from langchain.chains import GraphCypherQAChain

import embedding_handling
import file_handling
import graph_handling

In [2]:
graph = graph_handling.get_graph_connector()

In [3]:
graph.refresh_schema()

In [4]:
cypher_chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=graph_handling.GPT4,
    qa_llm=graph_handling.GPT4,
    validate_cypher=True, # Validate relationship directions
    verbose=True
)

In [7]:
cypher_chain.run("Where did Heinrich Schliemann discover?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Heinrich Schliemann"})-[:DISCOVERED]->(pl:Place) RETURN pl.name
Full Context:
[{'pl.name': 'Troy'}, {'pl.name': 'Mycenae'}]

> Finished chain.


'Heinrich Schliemann made his discoveries in Troy and Mycenae.'

In [11]:
print(graph.schema)


        Node properties are the following:
        [{'labels': 'Place', 'properties': [{'property': 'discoverydate', 'type': 'STRING'}, {'property': 'rebuildings', 'type': 'STRING'}, {'property': 'importance', 'type': 'STRING'}, {'property': 'id', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}, {'property': 'importantfeature', 'type': 'STRING'}, {'property': 'description', 'type': 'STRING'}, {'property': 'statehalves', 'type': 'STRING'}, {'property': 'regions', 'type': 'STRING'}, {'property': 'possiblestructure', 'type': 'STRING'}, {'property': 'destructiondate', 'type': 'STRING'}, {'property': 'alias', 'type': 'STRING'}, {'property': 'inhabitationperiod', 'type': 'STRING'}, {'property': 'significantchange', 'type': 'STRING'}, {'property': 'startofrecordedhistory', 'type': 'STRING'}, {'property': 'rapidexpansion', 'type': 'STRING'}, {'property': 'burntandlooteddate', 'type': 'STRING'}, {'property': 'lostrulerstatus', 'type': 'STRING'}, {'property': 'rulearea', 'type': 'STRI

In [12]:
cypher_chain.run("What are the IDs of the ObsidianNoteChunk nodes that have a relationship with the Mycenaeans?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:People {name: 'Mycenaeans'})-[:REFERENCES_NODE]-(o:ObsidianNoteChunk) RETURN o.id
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't know the answer."

In [1]:
import graph_handling

In [2]:
names = graph_handling.get_all_node_names()

In [3]:
names

['Greece',
 'Mycenae',
 'Absurdly Huge Stones And Giant Gravesites',
 'Tholos Tomb',
 'Knossos',
 'Wanax',
 'Mother Goddess',
 'Mycenaeanpalaces',
 'Heavilyarmoredsoldiers',
 'Kingsandcommanders',
 'Chariots',
 'Collapse',
 'Heinrich Schliemann',
 'Troy',
 'The Treasure Of Priam',
 'Mycenae',
 'Grave Circle A',
 'Minoans',
 'Crete',
 'Minoan Palaces',
 'Cretan Hieroglyphic Script',
 'Hieroglyphic Script From Phaistos',
 'Linear A',
 'Akrotiri',
 'Mycenaeans',
 'Kingdom Of Pylos',
 'Mycenaean Sparta',
 'Linear B Script Tablets',
 'Linear A',
 'Linear B',
 'Odyssey',
 'Near East',
 'Egypt',
 'Lerna',
 'House Of Tiles',
 'Minoan Civilization',
 'Crete',
 'Sir Arthur Evans',
 'Cnossus',
 'Anonymous Greek Historian',
 'The Parian Marble',
 'Franchthi Cave',
 'Greece',
 'Iliad']

In [4]:
relevant_names = graph_handling.get_relevant_nodes_from_question(graph_handling.GPT4, names, "Where did Heinrich Schliemann discover?", True)



> Entering new LLMChain chain...
Prompt after formatting:
System: # Prompt for GPT-4:
Question:
"Where did Heinrich Schliemann discover?"

List of Node Names from the Knowledge Graph:
Greece, Mycenae, Absurdly Huge Stones And Giant Gravesites, Tholos Tomb, Knossos, Wanax, Mother Goddess, Mycenaeanpalaces, Heavilyarmoredsoldiers, Kingsandcommanders, Chariots, Collapse, Heinrich Schliemann, Troy, The Treasure Of Priam, Mycenae, Grave Circle A, Minoans, Crete, Minoan Palaces, Cretan Hieroglyphic Script, Hieroglyphic Script From Phaistos, Linear A, Akrotiri, Mycenaeans, Kingdom Of Pylos, Mycenaean Sparta, Linear B Script Tablets, Linear A, Linear B, Odyssey, Near East, Egypt, Lerna, House Of Tiles, Minoan Civilization, Crete, Sir Arthur Evans, Cnossus, Anonymous Greek Historian, The Parian Marble, Franchthi Cave, Greece, Iliad

# Task for GPT-4:
Analyze the provided list of names from the knowledge graph in the context of the question. Identify and list the names that are most relevant t

In [5]:
relevant_names

NodeNameList(names=['Heinrich Schliemann', 'Troy', 'The Treasure Of Priam', 'Mycenae', 'Grave Circle A', 'Greece'])

In [6]:
connected_to_troy = graph_handling.get_graph_connector().query("""MATCH (n)
WHERE n.name = 'Troy'
OPTIONAL MATCH (n)-[r]-(related)
RETURN n, collect(r) as relationships, collect(related) as relatedNodes""")

In [25]:
connected_to_troy2 = graph_handling.get_graph_connector().query("""
MATCH (n)
WHERE n.name = 'Troy'
OPTIONAL MATCH (n)-[r]-(related)
RETURN n, 
       collect(r) as relationships, 
       collect(
           CASE WHEN 'ObsidianNoteChunk' IN labels(related) 
                THEN {label: 'ObsidianNoteChunk', file_name: related.file_name, chunk_number: related.chunk_number, id: related.id} 
                ELSE related 
           END
       ) as relatedNodes
""")

In [26]:
# nodes that have a relationship with the node Troy
connected_to_troy2[0]['relatedNodes']
# from here can grab id from nodes with label = ObsidianNoteChunk

# the relationships key can also be used to have GPT reason about the relationships between
# the Troy node and the other nodes (i.e. Heinrich Schliemann)

[{'profession': 'archaeologist',
  'characteristics': 'brilliant linguist, questionable, problematic',
  'name': 'Heinrich Schliemann',
  'id': 'Heinrich Schliemann'},
 {'id': 'b3c1a0e4-8eaf-47ed-b3d5-bf9953129af3',
  'chunk_number': 0,
  'label': 'ObsidianNoteChunk',
  'file_name': '09-02-23 - Lecture 2 - The Bronze Age.md'}]

In [12]:
def get_chunk_ids_by_node_names(node_names: list[str]) -> list[str]:
    """Given a list of node names, returns the ids of the chunks that reference them.
    
    May contain duplicates.
    """

    if len(node_names) == 0:
        return []

    ids: list[dict] = graph_handling.get_graph_connector().query(f"""
                            MATCH (n)
                            WHERE n.name IN [{",".join([f"'{name}'" for name in node_names])}]
                            OPTIONAL MATCH (n)-[r]-(related:ObsidianNoteChunk)""" +
                        """ RETURN collect({id: related.id}) as relatedNodes
                        """)

    return [list(d.values())[0] for d in ids[0]['relatedNodes']]

In [13]:
chunks_from_question = get_chunk_ids_by_node_names(relevant_names.names)

[{'relatedNodes': [{'id': 'f9453227-664b-41bb-9b9e-21fe3d5acfce'}, {'id': 'f9453227-664b-41bb-9b9e-21fe3d5acfce'}, {'id': 'bb0ab99e-debc-43ff-92f5-acca4167e99d'}, {'id': 'b3c1a0e4-8eaf-47ed-b3d5-bf9953129af3'}, {'id': 'b3c1a0e4-8eaf-47ed-b3d5-bf9953129af3'}, {'id': 'b3c1a0e4-8eaf-47ed-b3d5-bf9953129af3'}, {'id': 'b3c1a0e4-8eaf-47ed-b3d5-bf9953129af3'}, {'id': '139bea43-a0a8-4a4e-8a4d-6e8d8a7c954e'}, {'id': 'b3c1a0e4-8eaf-47ed-b3d5-bf9953129af3'}, {'id': 'c82fdc35-043c-4b58-8c20-0e97641a25ee'}]}]


In [15]:
set(chunks_from_question)

{'139bea43-a0a8-4a4e-8a4d-6e8d8a7c954e',
 'b3c1a0e4-8eaf-47ed-b3d5-bf9953129af3',
 'bb0ab99e-debc-43ff-92f5-acca4167e99d',
 'c82fdc35-043c-4b58-8c20-0e97641a25ee',
 'f9453227-664b-41bb-9b9e-21fe3d5acfce'}